# Analysis Notebook (Submission-ready)
This notebook reproduces the full workflow: data loading/cleaning, standard (2022–2023) analysis, market (2024) analysis, and exports to `outputs/`.

In [10]:
# --- Setup ---
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display

BASE_DIR = Path("D:/UWE CLASS/CSCT master/Project")
OUT_DIR = BASE_DIR / "outputs"
FIG_DIR = OUT_DIR / "figures"
OUT_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)

def std_ticker(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.upper()

def save_df(df: pd.DataFrame, name: str):
    xlsx = OUT_DIR / f"{name}.xlsx"
    csv  = OUT_DIR / f"{name}.csv"
    df.to_excel(xlsx, index=False)
    df.to_csv(csv, index=False)
    print("Saved:", xlsx)
    print("Saved:", csv)

def flatten_cols(df: pd.DataFrame) -> pd.DataFrame:
    # For groupby agg multiindex columns -> single line names (Excel friendly)
    out = df.copy()
    out.columns = [
        "_".join([str(x) for x in col if str(x) != ""]) if isinstance(col, tuple) else str(col)
        for col in out.columns
    ]
    return out


## 1) Company data (clean)

In [11]:
company_path = BASE_DIR / "Company.xlsx"
company_raw = pd.read_excel(company_path).copy()
company_raw["Ticker"] = std_ticker(company_raw["Ticker"])

# Drop non-analytical columns (keep evidence in report, not in dataset)
company = company_raw.drop(columns=["ESG Evidence Link", "Notes"], errors="ignore").copy()

save_df(company, "company_analysis")

print("Company shape:", company.shape)
display(company.head(10))


Saved: D:\UWE CLASS\CSCT master\Project\outputs\company_analysis.xlsx
Saved: D:\UWE CLASS\CSCT master\Project\outputs\company_analysis.csv
Company shape: (10, 7)


,Ticker,Company Name (SETTRADE),Market,Industry Sector,Company Website,Sustainability Disclosure Group,Listing Age (Years)
0,APO,Asian Palm Oil Public Company Limited,mai,Agro & Food Industry,asianpalmoil.com,0,0.75
1,TMILL,T.S. Flour Mill Public Company Limited,mai,Agro & Food Industry,www.tmill.co.th,1,12.10
2,AKP,AKKHIE PRATIKARN Public Company Limited,mai,Services,www.akkhie.com,1,11.90
3,PROEN,PROEN Corp Public Company Limited,mai,Technology,www.proen.co.th,1,3.67
4,PPS,P.P.S. Packaging Public Company Limited,mai,Property & Construction,www.pps.co.th,1,12.26
5,BSM,BSM Group Public Company Limited,mai,Property & Construction,www.bsmgroup.asia,1,16.88
6,CIG,C.I. Group Public Company Limited,mai,Industrial Products,www.cigpcl.com,0,19.93
7,MPJ,MPJ Logistics Public Company Limited,mai,Services,www.mpjlogistics.com,0,0.15
8,DOD,DOD Biotech Public Company Limited,mai,Consumer Products,www.dodbiotech.com,0,6.53
9,TNDT,Thai N.D.T. Public Company Limited,mai,Services,www.tndt.co.th,0,17.26


## 2) Financial data (clean)

In [12]:
fin_path = BASE_DIR / "Financial.xlsx"
fin_raw = pd.read_excel(fin_path).copy()

# Remove free-text source column (not used in analysis)
fin = fin_raw.drop(columns=["Source / Notes (URL, page, etc.)"], errors="ignore").copy()

# Standardize key fields
fin["Ticker"] = std_ticker(fin["Ticker"])
fin["Year"] = fin["Year"].astype(int)

save_df(fin, "financial_clean")

print("Financial shape:", fin.shape)
display(fin.head(10))


Saved: D:\UWE CLASS\CSCT master\Project\outputs\financial_clean.xlsx
Saved: D:\UWE CLASS\CSCT master\Project\outputs\financial_clean.csv
Financial shape: (30, 12)


,Ticker,Year,Use in main analysis? (Y/N),Revenue (THB mn) [Input],Net Profit (THB mn) [Input],Total Assets (THB mn) [Input],Total Liabilities (THB mn) [Input],Total Equity (THB mn) [Input],ROA [Formula],ROE [Formula],Net Profit Margin [Formula],D/E [Formula]
0,APO,2022,Y,2096.65,28.25,343.09,119.29,223.79,0.082340,0.126234,0.013474,0.533044
1,APO,2023,Y,1528.87,12.99,330.83,93.69,237.14,0.039265,0.054778,0.008496,0.395083
2,APO,2024,N,1798.11,107.06,473.03,87.58,385.44,0.226328,0.277760,0.059540,0.227221
3,TMILL,2022,Y,1966.04,109.40,2097.51,1012.06,1085.45,0.052157,0.100788,0.055645,0.932387
4,TMILL,2023,Y,1872.50,59.10,1725.64,664.81,1060.83,0.034248,0.055711,0.031562,0.626689
5,TMILL,2024,N,1721.57,19.19,1701.43,649.48,1051.95,0.011279,0.018242,0.011147,0.617406
6,AKP,2022,Y,384.19,29.29,727.53,124.20,603.33,0.040260,0.048547,0.076238,0.205857
7,AKP,2023,Y,372.33,16.86,716.55,106.99,609.56,0.023529,0.027659,0.045282,0.175520
8,AKP,2024,N,351.86,36.35,744.00,101.24,642.76,0.048858,0.056553,0.103308,0.157508
9,PROEN,2022,Y,1548.36,65.24,1590.38,1118.49,471.17,0.041022,0.138464,0.042135,2.373857


## 3) Main dataset (2022–2023) for standard analysis

In [13]:
# Filter rows used in main analysis (Y) -> expected Years: 2022–2023
fin_main = fin[fin["Use in main analysis? (Y/N)"].astype(str).str.upper().eq("Y")].copy()
print("fin_main shape:", fin_main.shape)
print("Years:", sorted(fin_main["Year"].unique()))

# Merge company fields (SR group, industry, listing age)
merged = fin_main.merge(
    company[["Ticker", "Sustainability Disclosure Group", "Industry Sector", "Listing Age (Years)"]],
    on="Ticker",
    how="left"
)

# Controls
merged["log_assets"] = np.log(merged["Total Assets (THB mn) [Input]"])

print("merged shape:", merged.shape)
print("Missing SR:", merged["Sustainability Disclosure Group"].isna().sum())
display(merged.head(10))


fin_main shape: (20, 12)
Years: [np.int64(2022), np.int64(2023)]
merged shape: (20, 16)
Missing SR: 0


,Ticker,Year,Use in main analysis? (Y/N),Revenue (THB mn) [Input],Net Profit (THB mn) [Input],Total Assets (THB mn) [Input],Total Liabilities (THB mn) [Input],Total Equity (THB mn) [Input],ROA [Formula],ROE [Formula],Net Profit Margin [Formula],D/E [Formula],Sustainability Disclosure Group,Industry Sector,Listing Age (Years),log_assets
0,APO,2022,Y,2096.65,28.25,343.09,119.29,223.79,0.082340,0.126234,0.013474,0.533044,0,Agro & Food Industry,0.75,5.837993
1,APO,2023,Y,1528.87,12.99,330.83,93.69,237.14,0.039265,0.054778,0.008496,0.395083,0,Agro & Food Industry,0.75,5.801605
2,TMILL,2022,Y,1966.04,109.40,2097.51,1012.06,1085.45,0.052157,0.100788,0.055645,0.932387,1,Agro & Food Industry,12.10,7.648506
3,TMILL,2023,Y,1872.50,59.10,1725.64,664.81,1060.83,0.034248,0.055711,0.031562,0.626689,1,Agro & Food Industry,12.10,7.453353
4,AKP,2022,Y,384.19,29.29,727.53,124.20,603.33,0.040260,0.048547,0.076238,0.205857,1,Services,11.90,6.589655
5,AKP,2023,Y,372.33,16.86,716.55,106.99,609.56,0.023529,0.027659,0.045282,0.175520,1,Services,11.90,6.574448
6,PROEN,2022,Y,1548.36,65.24,1590.38,1118.49,471.17,0.041022,0.138464,0.042135,2.373857,1,Technology,3.67,7.371728
7,PROEN,2023,Y,767.57,15.75,1801.13,1241.83,557.52,0.008745,0.028250,0.020519,2.227418,1,Technology,3.67,7.496170
8,PPS,2022,Y,438.84,22.77,676.85,345.95,333.69,0.033641,0.068237,0.051887,1.036741,1,Property & Construction,12.26,6.517450
9,PPS,2023,Y,436.63,-18.19,617.61,306.50,313.39,-0.029452,-0.058043,-0.041660,0.978015,1,Property & Construction,12.26,6.425857


## 4) Descriptive statistics (standard, by SR)

In [14]:
metrics = ["ROA [Formula]", "ROE [Formula]", "Net Profit Margin [Formula]", "D/E [Formula]"]

desc = merged.groupby("Sustainability Disclosure Group")[metrics].agg(["count","mean","std","min","median","max"])
display(desc)

desc_flat = flatten_cols(desc.reset_index())
desc_flat.to_excel(OUT_DIR / "desc_by_SR_flat.xlsx", index=False)
print("Saved:", OUT_DIR / "desc_by_SR_flat.xlsx")


ROA [Formula]                                \
                                        count      mean       std       min   
Sustainability Disclosure Group                                               
0                                          10 -0.069152  0.202033 -0.512128   
1                                          10  0.013830  0.033776 -0.042557   

                                                    ROE [Formula]            \
                                   median       max         count      mean   
Sustainability Disclosure Group                                               
0                                0.004768  0.128853            10 -0.383593   
1                                0.028585  0.052157            10  0.021286   

                                                     ...  \
                                      std       min  ...   
Sustainability Disclosure Group                      ...   
0                                0.997372 -2.970036  ...   
1                                0.082245 -0.127304  ...   

                                Net Profit Margin [Formula]            \
                                                        std       min   
Sustainability Disclosure Group                                         
0                                                  0.415089 -1.166884   
1                                                  0.057873 -0.107417   

                                                    D/E [Formula]            \
                                   median       max         count      mean   
Sustainability Disclosure Group                                               
0                                0.008947  0.089014            10  1.843671   
1                                0.036848  0.076238            10  1.249987   

                                                                         
                                      std       min    median       max  
Sustainability Disclosure Group                                          
0                                2.039279  0.356901  0.909224  5.944623  
1                                0.824207  0.175520  1.007378  2.373857  

[2 rows x 24 columns]

Saved: D:\UWE CLASS\CSCT master\Project\outputs\desc_by_SR_flat.xlsx


## 5) t-tests (standard)
We report both **pooled-variance** t-test and **Welch** t-test.

In [15]:
from scipy import stats

rows=[]
for m in metrics:
    g1 = merged.loc[merged["Sustainability Disclosure Group"]==1, m].dropna()
    g0 = merged.loc[merged["Sustainability Disclosure Group"]==0, m].dropna()

    t_pooled, p_pooled = stats.ttest_ind(g1, g0, equal_var=True)
    t_welch,  p_welch  = stats.ttest_ind(g1, g0, equal_var=False)

    # effect size (Cohen's d)
    n1, n0 = len(g1), len(g0)
    s1, s0 = g1.std(ddof=1), g0.std(ddof=1)
    sp = np.sqrt(((n1-1)*s1**2 + (n0-1)*s0**2) / (n1+n0-2)) if (n1+n0-2)>0 else np.nan
    d = (g1.mean() - g0.mean())/sp if sp and sp!=0 else np.nan

    rows.append({
        "Metric": m,
        "Mean_SR1": g1.mean(), "Mean_SR0": g0.mean(),
        "n_SR1": n1, "n_SR0": n0,
        "t_pooled": t_pooled, "p_pooled": p_pooled,
        "t_welch": t_welch, "p_welch": p_welch,
        "cohens_d": d
    })

ttest_standard = pd.DataFrame(rows)
display(ttest_standard)

ttest_standard.to_excel(OUT_DIR / "ttest_standard_enhanced.xlsx", index=False)
print("Saved:", OUT_DIR / "ttest_standard_enhanced.xlsx")


,Metric,Mean_SR1,Mean_SR0,n_SR1,n_SR0,t_pooled,p_pooled,t_welch,p_welch,cohens_d
0,ROA [Formula],0.013830,-0.069152,10,10,1.281064,0.216431,1.281064,0.230550,0.572909
1,ROE [Formula],0.021286,-0.383593,10,10,1.279371,0.217013,1.279371,0.232347,0.572152
2,Net Profit Margin [Formula],0.013211,-0.203282,10,10,1.633511,0.119730,1.633511,0.135530,0.730528
3,D/E [Formula],1.249987,1.843671,10,10,-0.853539,0.404576,-0.853539,0.410263,-0.381714


Saved: D:\UWE CLASS\CSCT master\Project\outputs\ttest_standard_enhanced.xlsx


## 6) Regression (standard)
OLS with robust (HC3) standard errors.

In [16]:
import statsmodels.formula.api as smf

reg = merged.rename(columns={"Sustainability Disclosure Group":"SR"}).copy()
reg["SR"] = reg["SR"].astype(int)

# Fit models (HC3 robust SE)
m_roa = smf.ols("Q('ROA [Formula]') ~ SR + log_assets + Q('D/E [Formula]') + C(Year)", data=reg).fit(cov_type="HC3")
m_roe = smf.ols("Q('ROE [Formula]') ~ SR + log_assets + Q('D/E [Formula]') + C(Year)", data=reg).fit(cov_type="HC3")
m_npm = smf.ols("Q('Net Profit Margin [Formula]') ~ SR + log_assets + Q('D/E [Formula]') + C(Year)", data=reg).fit(cov_type="HC3")

def coef_table(model, name):
    t = pd.DataFrame({
        "model": name,
        "term": model.params.index,
        "coef": model.params.values,
        "std_err": model.bse.values,
        "t": model.tvalues.values,
        "p_value": model.pvalues.values
    })
    t["N"]=int(model.nobs)
    t["R2"]=model.rsquared
    t["Adj_R2"]=model.rsquared_adj
    return t

reg_table = pd.concat([coef_table(m_roa,"ROA"), coef_table(m_roe,"ROE"), coef_table(m_npm,"NPM")], ignore_index=True)
display(reg_table)

reg_table.to_excel(OUT_DIR / "regression_standard_HC3.xlsx", index=False)
print("Saved:", OUT_DIR / "regression_standard_HC3.xlsx")


,model,term,coef,std_err,t,p_value,N,R2,Adj_R2
0,ROA,Intercept,0.155884,0.354501,0.439729,0.660134,20,0.348889,0.175259
1,ROA,C(Year)[T.2023],-0.032765,0.083186,-0.393873,0.693675,20,0.348889,0.175259
2,ROA,SR,0.060030,0.080854,0.742446,0.457817,20,0.348889,0.175259
3,ROA,log_assets,-0.017738,0.065168,-0.272186,0.785479,20,0.348889,0.175259
4,ROA,Q('D/E [Formula]'),-0.048658,0.044946,-1.082582,0.278994,20,0.348889,0.175259
5,ROE,Intercept,-0.633486,1.297100,-0.488386,0.625276,20,0.565257,0.449325
6,ROE,C(Year)[T.2023],-0.256923,0.351384,-0.731174,0.464673,20,0.565257,0.449325
7,ROE,SR,0.162329,0.239786,0.676973,0.498423,20,0.565257,0.449325
8,ROE,log_assets,0.146107,0.226214,0.645878,0.518358,20,0.565257,0.449325
9,ROE,Q('D/E [Formula]'),-0.326186,0.218599,-1.492166,0.135656,20,0.565257,0.449325


Saved: D:\UWE CLASS\CSCT master\Project\outputs\regression_standard_HC3.xlsx


## 7) Market variables dataset (2024) + tests/regression

In [17]:
# Prepare 2024 financial + company
fin_2024 = fin[fin["Year"].eq(2024)].copy()
fin_2024 = fin_2024.merge(
    company[["Ticker","Sustainability Disclosure Group","Industry Sector","Listing Age (Years)"]],
    on="Ticker",
    how="left"
)
fin_2024["log_assets"] = np.log(fin_2024["Total Assets (THB mn) [Input]"])
print("fin_2024 shape:", fin_2024.shape, "Missing SR:", fin_2024["Sustainability Disclosure Group"].isna().sum())

# Load market variables (2024)
mkt_path = BASE_DIR / "Market_variables.xlsx"
mkt = pd.read_excel(mkt_path).drop(columns=["Source / Notes"], errors="ignore").copy()
mkt["Ticker"] = std_ticker(mkt["Ticker"])
mkt["Year"] = mkt["Year"].astype(int)

dataset_2024 = fin_2024.merge(mkt, on=["Ticker","Year"], how="left")
print("dataset_2024 shape:", dataset_2024.shape)
display(dataset_2024.head(10))


fin_2024 shape: (10, 16) Missing SR: 0
dataset_2024 shape: (10, 23)


,Ticker,Year,Use in main analysis? (Y/N)_x,Revenue (THB mn) [Input],Net Profit (THB mn) [Input],Total Assets (THB mn) [Input],Total Liabilities (THB mn) [Input],Total Equity (THB mn) [Input],ROA [Formula],ROE [Formula],...,Industry Sector,Listing Age (Years),log_assets,Use in main analysis? (Y/N)_y,Market Cap (THB mn) [Input],Year-end Close Price (THB) [Input],Annual Return % [Formula/Optional],P/E (x) [Input],P/BV (x) [Input],Avg Daily Value (THB mn) [Input]
0,APO,2024,N,1798.11,107.06,473.03,87.58,385.44,0.226328,0.277760,...,Agro & Food Industry,0.75,6.159159,Y,601.80,1.77,NaN,3.42,1.22,NaN
1,TMILL,2024,N,1721.57,19.19,1701.43,649.48,1051.95,0.011279,0.018242,...,Agro & Food Industry,12.10,7.439224,Y,837.19,2.10,NaN,21.03,0.79,NaN
2,AKP,2024,N,351.86,36.35,744.00,101.24,642.76,0.048858,0.056553,...,Services,11.90,6.612041,Y,242.40,0.60,NaN,3.99,0.36,NaN
3,PROEN,2024,N,716.38,29.23,1470.41,890.59,579.79,0.019879,0.050415,...,Technology,3.67,7.293297,Y,423.97,1.08,NaN,4.84,0.71,NaN
4,PPS,2024,N,386.59,-14.18,589.25,294.11,296.54,-0.024064,-0.047818,...,Property & Construction,12.26,6.378851,Y,146.18,0.17,NaN,NaN,0.52,NaN
5,BSM,2024,N,489.97,-78.67,1086.54,739.41,333.05,-0.072404,-0.236211,...,Property & Construction,16.88,6.990754,Y,78.83,0.37,NaN,NaN,0.26,NaN
6,CIG,2024,N,445.32,-432.88,892.09,442.93,446.94,-0.485243,-0.968542,...,Industrial Products,19.93,6.793567,Y,112.10,0.03,NaN,NaN,0.32,NaN
7,MPJ,2024,N,1048.64,99.98,1213.87,521.25,688.01,0.082365,0.145318,...,Services,0.15,7.101569,Y,696.00,3.48,NaN,5.79,0.97,NaN
8,DOD,2024,N,662.77,-159.33,1286.31,394.40,898.60,-0.123866,-0.177309,...,Consumer Products,6.53,7.159533,Y,729.67,1.62,NaN,14.31,0.77,NaN
9,TNDT,2024,N,301.66,-274.37,691.62,570.30,126.32,-0.396706,-2.172023,...,Services,17.26,6.539037,Y,227.05,0.16,NaN,NaN,2.85,NaN


In [18]:
# Descriptive (market 2024) by SR


dataset_2024.columns = dataset_2024.columns.str.strip()

market_vars = [
    "Market Cap (THB mn) [Input]",
    "Year-end Close Price (THB) [Input]",
    "P/E (x) [Input]",          
    "P/BV (x) [Input]"
]

desc_mkt = (
    dataset_2024
    .groupby("Sustainability Disclosure Group")[market_vars]
    .agg(["count","mean","std","min","median","max"])
)
display(desc_mkt)

desc_mkt_flat = flatten_cols(desc_mkt.reset_index())
desc_mkt_flat.to_excel(OUT_DIR / "desc_market_2024_by_SR_flat.xlsx", index=False)
print("Saved:", OUT_DIR / "desc_market_2024_by_SR_flat.xlsx")


Market Cap (THB mn) [Input]           \
                                                      count     mean   
Sustainability Disclosure Group                                        
0                                                         5  473.324   
1                                                         5  345.714   

                                                                    \
                                        std     min median     max   
Sustainability Disclosure Group                                      
0                                284.138002  112.10  601.8  729.67   
1                                303.886216   78.83  242.4  837.19   

                                Year-end Close Price (THB) [Input]         \
                                                             count   mean   
Sustainability Disclosure Group                                             
0                                                                5  1.412   
1                                                                5  0.864   

                                                 ... P/E (x) [Input]        \
                                      std   min  ...             std   min   
Sustainability Disclosure Group                  ...                         
0                                1.407611  0.03  ...        5.727120  3.42   
1                                0.769695  0.17  ...        9.602085  3.99   

                                              P/BV (x) [Input]         \
                                median    max            count   mean   
Sustainability Disclosure Group                                         
0                                 5.79  14.31                5  1.226   
1                                 4.84  21.03                5  0.528   

                                                              
                                      std   min median   max  
Sustainability Disclosure Group                               
0                                0.965883  0.32   0.97  2.85  
1                                0.224655  0.26   0.52  0.79  

[2 rows x 24 columns]

Saved: D:\UWE CLASS\CSCT master\Project\outputs\desc_market_2024_by_SR_flat.xlsx


In [19]:
# t-tests (market 2024): pooled + Welch + effect size
from scipy import stats

rows=[]
for v in market_vars:
    g1 = dataset_2024.loc[dataset_2024["Sustainability Disclosure Group"]==1, v].dropna()
    g0 = dataset_2024.loc[dataset_2024["Sustainability Disclosure Group"]==0, v].dropna()

    if len(g1)<2 or len(g0)<2:
        rows.append({"Variable": v, "note":"Not enough data for t-test", "n_SR1":len(g1), "n_SR0":len(g0)})
        continue

    t_pooled, p_pooled = stats.ttest_ind(g1, g0, equal_var=True)
    t_welch,  p_welch  = stats.ttest_ind(g1, g0, equal_var=False)

    n1, n0 = len(g1), len(g0)
    s1, s0 = g1.std(ddof=1), g0.std(ddof=1)
    sp = np.sqrt(((n1-1)*s1**2 + (n0-1)*s0**2) / (n1+n0-2)) if (n1+n0-2)>0 else np.nan
    d = (g1.mean() - g0.mean())/sp if sp and sp!=0 else np.nan

    rows.append({
        "Variable": v,
        "Mean_SR1": g1.mean(), "Mean_SR0": g0.mean(),
        "n_SR1": n1, "n_SR0": n0,
        "t_pooled": t_pooled, "p_pooled": p_pooled,
        "t_welch": t_welch, "p_welch": p_welch,
        "cohens_d": d
    })

ttest_mkt = pd.DataFrame(rows)
display(ttest_mkt)

ttest_mkt.to_excel(OUT_DIR / "ttest_market_2024_enhanced.xlsx", index=False)
print("Saved:", OUT_DIR / "ttest_market_2024_enhanced.xlsx")


,Variable,Mean_SR1,Mean_SR0,n_SR1,n_SR0,t_pooled,p_pooled,t_welch,p_welch,cohens_d
0,Market Cap (THB mn) [Input],345.714000,473.324,5,5,-0.685875,0.512175,-0.685875,0.512260,-0.433785
1,Year-end Close Price (THB) [Input],0.864000,1.412,5,5,-0.763797,0.466914,-0.763797,0.473062,-0.483068
2,P/E (x) [Input],9.953333,7.840,3,3,0.327396,0.759786,0.327396,0.763262,0.267318
3,P/BV (x) [Input],0.528000,1.226,5,5,-1.573893,0.154159,-1.573893,0.183712,-0.995418


Saved: D:\UWE CLASS\CSCT master\Project\outputs\ttest_market_2024_enhanced.xlsx


In [20]:
# Regression (market 2024) with HC3 robust SE
import statsmodels.formula.api as smf

dfm = dataset_2024.copy()
dfm["SR"] = dfm["Sustainability Disclosure Group"].astype(int)

dfm["log_mcap"] = np.log(dfm["Market Cap (THB mn) [Input]"])
dfm["close_price"] = pd.to_numeric(dfm["Year-end Close Price (THB) [Input]"], errors="coerce")
dfm.loc[dfm["close_price"]<=0, "close_price"] = np.nan
dfm["log_close"] = np.log(dfm["close_price"])

dfm["pbv"] = pd.to_numeric(dfm["P/BV (x) [Input]"], errors="coerce")
dfm["de"]  = pd.to_numeric(dfm["D/E [Formula]"], errors="coerce")

# log_assets already exists from fin_2024 merge
m1 = smf.ols("log_mcap ~ SR + log_assets + de", data=dfm.dropna(subset=["log_mcap","SR","log_assets","de"])).fit(cov_type="HC3")
m2 = smf.ols("pbv ~ SR + log_assets + de", data=dfm.dropna(subset=["pbv","SR","log_assets","de"])).fit(cov_type="HC3")
m3 = smf.ols("log_close ~ SR + log_assets + de", data=dfm.dropna(subset=["log_close","SR","log_assets","de"])).fit(cov_type="HC3")

reg_mkt = pd.concat([
    coef_table(m1,"log(MarketCap)"),
    coef_table(m2,"P/BV"),
    coef_table(m3,"log(ClosePrice)")
], ignore_index=True)

display(reg_mkt)
reg_mkt.to_excel(OUT_DIR / "regression_market_2024_HC3.xlsx", index=False)
print("Saved:", OUT_DIR / "regression_market_2024_HC3.xlsx")


,model,term,coef,std_err,t,p_value,N,R2,Adj_R2
0,log(MarketCap),Intercept,0.680850,5.578658,0.122045,0.902863,10,0.401266,0.101899
1,log(MarketCap),SR,-0.646358,0.683832,-0.945199,0.344557,10,0.401266,0.101899
2,log(MarketCap),log_assets,0.832737,0.797299,1.044448,0.296278,10,0.401266,0.101899
3,log(MarketCap),de,-0.263957,0.748323,-0.352731,0.724290,10,0.401266,0.101899
4,P/BV,Intercept,2.123419,3.434208,0.618314,0.536368,10,0.687646,0.531468
5,P/BV,SR,-0.549789,0.473383,-1.161403,0.245478,10,0.687646,0.531468
6,P/BV,log_assets,-0.211435,0.510636,-0.414062,0.678829,10,0.687646,0.531468
7,P/BV,de,0.382341,0.578748,0.660635,0.508846,10,0.687646,0.531468
8,log(ClosePrice),Intercept,-8.790167,10.599272,-0.829318,0.406925,10,0.293697,-0.059454
9,log(ClosePrice),SR,-0.257969,1.145885,-0.225127,0.821881,10,0.293697,-0.059454


Saved: D:\UWE CLASS\CSCT master\Project\outputs\regression_market_2024_HC3.xlsx


## 8) Figures (market 2024)
Boxplots and sorted-rank line plots (saved to `outputs/figures/`).

In [21]:
import matplotlib.pyplot as plt

plot_vars = [
    "Market Cap (THB mn) [Input]",
    "Year-end Close Price (THB) [Input]",
    "P/BV (x) [Input]"
]

# Boxplots
for v in plot_vars:
    ax = dataset_2024.boxplot(column=v, by="Sustainability Disclosure Group")
    plt.title(f"{v} (2024) by SR Group")
    plt.suptitle("")
    plt.xlabel("SR group (0=No report, 1=Report)")
    plt.ylabel(v)
    plt.tight_layout()
    out = FIG_DIR / f"boxplot_2024_{v.replace(' ','_').replace('/','_')}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight")
    plt.close()

# Sorted line plots (rank within SR group)
for v in plot_vars:
    tmp = dataset_2024[["Sustainability Disclosure Group", v]].dropna()
    g0 = tmp[tmp["Sustainability Disclosure Group"]==0][v].sort_values().reset_index(drop=True)
    g1 = tmp[tmp["Sustainability Disclosure Group"]==1][v].sort_values().reset_index(drop=True)

    plt.figure()
    plt.plot(range(1,len(g0)+1), g0.values, marker="o", label="SR=0")
    plt.plot(range(1,len(g1)+1), g1.values, marker="o", label="SR=1")
    plt.title(f"Sorted distribution (2024): {v}")
    plt.xlabel("Rank within SR group (low → high)")
    plt.ylabel(v)
    plt.legend()
    plt.tight_layout()
    out = FIG_DIR / f"sortedline_2024_{v.replace(' ','_').replace('/','_')}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight")
    plt.close()

print("Saved figures to:", FIG_DIR)


Saved figures to: D:\UWE CLASS\CSCT master\Project\outputs\figures
